In [21]:
import glob
import json
import shutil
from datetime import date, timedelta
from pathlib import Path

import boto3
import pandas as pd
from botocore.config import Config
from botocore.exceptions import NoCredentialsError

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

In [32]:
def export_scenes_list(many_scenes, batch_tag, batch_size=100):
    batch_num = len(many_scenes) // batch_size + 1
    if len(many_scenes) % batch_size == 0:
        batch_num -= 1
    import os

    dest = f"../../data/landsat/task/{batch_tag}"
    try:
        os.makedirs(dest)
    except FileExistsError:
        # directory already exists
        pass
    for i in range(1, batch_num + 1):
        with open(f"{dest}/scenes_{str(i).zfill(4)}.txt", "w") as fp:
            fp.write("landsat_ot_c2_l1|displayId\n")
            for item in many_scenes[(i - 1) * batch_size : i * batch_size]:
                # write each item on a new line
                fp.write(f"{item}\n")
    print("Done", batch_num)


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)


def func_over_daterange(start_date, end_date, func):
    for single_date in daterange(start_date, end_date):
        print(single_date.strftime("%Y%m%d"))
        # call daily function
        func(single_date.strftime("%Y%m%d"))

In [4]:
ENDPOINT = os.getenv("PIKNIK_ENDPOINT")
PROVIDER = os.getenv("PIKNIK_PROVIDER")

In [5]:
ACCESS_KEY = os.getenv("PIKNIK_ACCESS_KEY_ID")
SECRET_KEY = os.getenv("PIKNIK_SECRET_ACCESS_KEY")

In [6]:
s3 = boto3.resource(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url="http://" + ENDPOINT,
)

In [7]:
my_bucket = s3.Bucket("uofm-data1")

In [8]:
#### Upload missing files

In [11]:
with open("../../data/temp/missing_files_dump.json", "w") as fp:
    json.dump(missing_files, fp)

NameError: name 'missing_files' is not defined

In [21]:
with open("../../data/temp/missing_files_dump.json", "r") as fp:
    print(fp.readlines())

[]


In [19]:
with open("../../data/temp/missing_files_dump.json", "r") as fp:

    if len(fp.readlines()) != 0:
        print(fp.readlines())
        fp.seek(0)
        missing_files = json.load(fp)

In [18]:
missing_files

NameError: name 'missing_files' is not defined

In [173]:
[(k, missing_files[k]) for k in missing_files.keys() if get_dt_str(k) == "20211103"]

[('LC09_L1GT_112068_20211103_20220118_02_T2',
  ['LC09_L1GT_112068_20211103_20220118_02_T2_B6.TIF']),
 ('LC09_L1TP_192052_20211103_20220118_02_T1',
  ['LC09_L1TP_192052_20211103_20220118_02_T1_thumb_small.jpeg',
   'LC09_L1TP_192052_20211103_20220118_02_T1_stac.json',
   'LC09_L1TP_192052_20211103_20220118_02_T1_thumb_large.jpeg']),
 ('LC09_L1TP_192053_20211103_20220118_02_T1',
  ['LC09_L1TP_192053_20211103_20220118_02_T1_stac.json',
   'LC09_L1TP_192053_20211103_20220118_02_T1_thumb_large.jpeg',
   'LC09_L1TP_192053_20211103_20220118_02_T1_thumb_small.jpeg'])]

In [55]:
list(
    my_bucket.objects.filter(
        Prefix="landsat9-c2-l1/20211103/LC09_L1GT_112068_20211103_20220118_02_T2/LC09_L1GT_112068_20211103_20220118_02_T2_B5.TIF"
    )
)

[s3.ObjectSummary(bucket_name='uofm-data1', key='landsat9-c2-l1/20211103/LC09_L1GT_112068_20211103_20220118_02_T2/LC09_L1GT_112068_20211103_20220118_02_T2_B5.TIF')]

In [25]:
len(list(my_bucket.objects.filter(Prefix="landsat9-c2-l1-missing")))

9432

[('LC09_L1TP_054023_20220101_20220123_02_T1',
  ['LC09_L1TP_054023_20220101_20220123_02_T1_B4.TIF']),
 ('LC09_L1GT_111059_20220101_20220121_02_T2',
  ['LC09_L1GT_111059_20220101_20220121_02_T2_thumb_small.jpeg']),
 ('LC09_L1GT_095122_20220101_20220122_02_T2',
  ['LC09_L1GT_095122_20220101_20220122_02_T2_thumb_small.jpeg']),
 ('LC09_L1TP_111036_20220101_20220121_02_T1',
  ['LC09_L1TP_111036_20220101_20220121_02_T1_QA_RADSAT.TIF',
   'LC09_L1TP_111036_20220101_20220121_02_T1_SZA.TIF',
   'LC09_L1TP_111036_20220101_20220121_02_T1_SAA.TIF'])]

In [28]:
# for s,fl in tqdm.tqdm(missing_files.items()):
#     for f in fl:
#         local_file = f'../../data/landsat/m2m_download/{get_dt_str(s)}/{s}/{f}'
#         remote_key = f'landsat9-c2-l1/{get_dt_str(s)}/{s}/{f}'
#         if os.path.exists(local_file):
#             s3.meta.client.upload_file(local_file, 'uofm-data1', remote_key)

In [57]:
#### Checking integrity of S3

In [9]:
missing_files = {}
scene_cnt = 0
file_cnt = 0

In [10]:
def asset_helper(href):
    asset = href.split("/")[-1]
    if "." in asset:
        return asset
    else:
        return asset + "_stac.json"

In [11]:
import tqdm

In [26]:
# remote_files = [obj.key.split('/')[-1] for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/20211103")]

In [12]:
def check_s3_one_day(cur_date):
    datestr = cur_date.strftime("%Y%m%d")
    remote_files = [
        obj.key.split("/")[-1]
        for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/{datestr}")
    ]
    stac_list = glob.glob(f"../../data/stac/landsat-c2l1/json/{datestr}/*.json")
    scene_cnt = 0
    file_cnt = 0
    for sf in stac_list:
        with open(sf, "r") as f:
            import json

            stac_objects = json.load(f)

        diff = set(
            [asset_helper(obj["href"]) for k, obj in stac_objects["assets"].items()]
        ) - set(remote_files)
        if diff:
            missing_files[stac_objects["id"]] = list(diff)
            scene_cnt += 1
            file_cnt += len(diff)

    print("checked", datestr, scene_cnt, file_cnt)

In [40]:
check_s3_one_day(date(2021, 11, 3))

checked 20211103 3 7


In [ ]:
# from multiprocessing import Pool
# with Pool(20) as p:
#     print(p.map(check_s3_one_day, daterange(start_date, end_date)))

In [49]:
start_date = date(2021, 10, 31)
end_date = date(2022, 11, 1)

for cur_date in tqdm.tqdm(
    daterange(start_date, end_date), total=(end_date - start_date).days
):
    datestr = cur_date.strftime("%Y%m%d")
    remote_files = [
        obj.key.split("/")[-1]
        for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/{datestr}")
    ]
    stac_list = glob.glob(f"../../data/stac/landsat-c2l1/json/{datestr}/*.json")

    for sf in stac_list:
        with open(sf, "r") as f:
            import json

            stac_objects = json.load(f)

        diff = set(
            [asset_helper(obj["href"]) for k, obj in stac_objects["assets"].items()]
        ) - set(remote_files)
        if diff:
            missing_files[stac_objects["id"]] = list(diff)
            scene_cnt += 1
            file_cnt += len(diff)
    # print(datestr,len(remote_files),scene_cnt,file_cnt)

100%|█████████████████████████████████████████████████████| 366/366 [1:33:33<00:00, 15.34s/it]


In [64]:
excluding = [
    "LC09_L1TP_108078_20220520_20220520_02_T1_SAA.TIF",
    "LC09_L1TP_108082_20220520_20220520_02_T1_VZA.TIF",
    "LC09_L1TP_108083_20220520_20220520_02_T1_MTL.txt",
    "LC09_L1GT_108057_20220520_20220520_02_T2_ANG.txt",
    "LC09_L1TP_108233_20220520_20220520_02_T2_SZA.TIF",
    "LC09_L1TP_108062_20220520_20220520_02_T1_VAA.TIF",
    "LC09_L1TP_108070_20220520_20220520_02_T1_MTL.xml",
]

In [16]:
with open("../../data/output/missing_s3.txt", "r") as fr:
    missing_file_keys = fr.readlines()

In [19]:
missing_file_keys = [k.replace("\n", "") for k in missing_file_keys]

In [23]:
for k in missing_file_keys:
    shutil.copy(
        "../../data/landsat/" + k,
        "../../data/landsat/landsat9-c2-l1-missing/" + k.split("/")[-1],
    )

In [66]:
with open("../../data/output/missing_s3.txt", "w") as fp:
    for scene in missing_files:
        get_dt_str = lambda p: p.split("_")[3]
        for asset in missing_files[scene]:
            if asset not in excluding:
                destination = f"landsat9-c2-l1/{get_dt_str(scene)}/{scene}/{asset}"
                fp.write(destination)
                fp.write("\n")

In [62]:
for scene in missing_files:
    get_dt_str = lambda p: p.split("_")[3]
    for asset in missing_files[scene]:
        source = f"../../data/landsat/m2m_download/{get_dt_str(scene)}/{scene}/{asset}"
        destination = (
            f"../../data/landsat/landsat9-c2-l1/{get_dt_str(scene)}/{scene}/{asset}"
        )
        os.makedirs(os.path.dirname(destination), exist_ok=True)
        try:
            shutil.copy(source, destination)
            # print(f"File {asset} copied successfully.")

        # For other errors
        except Exception as e:
            print(f"Error {e} occurred while copying file {asset}")

Error [Errno 2] No such file or directory: '../../data/landsat/m2m_download/20220520/LC09_L1TP_108078_20220520_20220520_02_T1/LC09_L1TP_108078_20220520_20220520_02_T1_SAA.TIF' occurred while copying file LC09_L1TP_108078_20220520_20220520_02_T1_SAA.TIF
Error [Errno 2] No such file or directory: '../../data/landsat/m2m_download/20220520/LC09_L1TP_108082_20220520_20220520_02_T1/LC09_L1TP_108082_20220520_20220520_02_T1_VZA.TIF' occurred while copying file LC09_L1TP_108082_20220520_20220520_02_T1_VZA.TIF
Error [Errno 2] No such file or directory: '../../data/landsat/m2m_download/20220520/LC09_L1TP_108083_20220520_20220520_02_T1/LC09_L1TP_108083_20220520_20220520_02_T1_MTL.txt' occurred while copying file LC09_L1TP_108083_20220520_20220520_02_T1_MTL.txt
Error [Errno 2] No such file or directory: '../../data/landsat/m2m_download/20220520/LC09_L1GT_108057_20220520_20220520_02_T2/LC09_L1GT_108057_20220520_20220520_02_T2_ANG.txt' occurred while copying file LC09_L1GT_108057_20220520_20220520_02

In [ ]:
missing_files

In [50]:
scene_cnt, file_cnt

(2180, 9439)

In [99]:
scene_cnt, file_cnt

(2180, 9439)

In [77]:
sf

'../../data/stac/landsat-c2l1/json/20211211/LC09_L1GT_220105_20211211_20220120_02_T2_stac.json'

In [103]:
export_scenes_list(list(missing_files.keys()), "batch_ms1", 200)

Done 11


In [ ]:
missing_files

In [ ]:
# LC09_L1GT_068110_20220916_20220916_02_T2

In [52]:
get_dt_str = lambda p: p.split("_")[3]

In [119]:
m_pid = []

In [122]:
for pid in list(missing_files.keys()):
    # pid = "LC09_L1GT_173020_20220916_20220916_02_T2"
    f_cached = [
        f.split("/")[-1]
        for f in glob.glob(f"../../data/landsat/m2m_download/{get_dt_str(pid)}/{pid}/*")
    ]
    if set(missing_files[pid]) - set(f_cached):
        m_pid.append(pid)
        print(set(missing_files[pid]) - set(f_cached))

{'LC09_L1TP_108078_20220520_20220520_02_T1_SAA.TIF'}
{'LC09_L1TP_108082_20220520_20220520_02_T1_VZA.TIF'}
{'LC09_L1TP_108083_20220520_20220520_02_T1_MTL.txt'}
{'LC09_L1GT_108057_20220520_20220520_02_T2_ANG.txt'}
{'LC09_L1TP_108233_20220520_20220520_02_T2_SZA.TIF'}
{'LC09_L1TP_108062_20220520_20220520_02_T1_VAA.TIF'}
{'LC09_L1TP_108070_20220520_20220520_02_T1_MTL.xml'}


In [121]:
m_pid

['LC09_L1TP_108078_20220520_20220520_02_T1',
 'LC09_L1TP_108082_20220520_20220520_02_T1',
 'LC09_L1TP_108083_20220520_20220520_02_T1',
 'LC09_L1GT_108057_20220520_20220520_02_T2',
 'LC09_L1TP_108233_20220520_20220520_02_T2',
 'LC09_L1TP_108062_20220520_20220520_02_T1',
 'LC09_L1TP_108070_20220520_20220520_02_T1',
 'LC09_L1TP_200044_20221015_20221015_02_T1',
 'LC09_L1TP_045016_20221017_20221018_02_T1',
 'LC09_L1TP_045014_20221017_20221018_02_T1',
 'LC09_L1TP_029047_20221017_20221017_02_T1',
 'LC09_L1TP_029041_20221017_20221017_02_T1',
 'LC09_L1TP_029035_20221017_20221017_02_T1',
 'LC09_L1TP_029044_20221017_20221017_02_T1',
 'LC09_L1TP_029033_20221017_20221017_02_T1',
 'LC09_L1TP_045007_20221017_20221018_02_T1',
 'LC09_L1TP_029046_20221017_20221017_02_T1',
 'LC09_L1TP_029042_20221017_20221017_02_T1',
 'LC09_L1TP_029043_20221017_20221017_02_T1',
 'LC09_L1TP_029036_20221017_20221017_02_T1',
 'LC09_L1TP_045008_20221017_20221018_02_T2',
 'LC09_L1TP_029037_20221017_20221017_02_T1',
 'LC09_L1T

In [104]:
missing_files["LC09_L1GT_068110_20220916_20220916_02_T2"]

['LC09_L1GT_068110_20220916_20220916_02_T2_B4.TIF']

In [74]:
datestr

'20211211'

In [68]:
datestr = "20220430"


remote_files = [
    obj.key.split("/")[-1]
    for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/{datestr}")
]

In [55]:
datestr = "20220430"


remote_files = [
    obj.key.split("/")[-1]
    for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/{datestr}")
]

import glob

stac_list = glob.glob(f"../../data/stac/landsat-c2l1/json/{datestr}/*.json")


def asset_helper(href):
    asset = href.split("/")[-1]
    if "." in asset:
        return asset
    else:
        return asset + "_stac.json"


for sf in stac_list:
    with open(sf, "r") as f:
        import json

        stac_objects = json.load(f)
    diff = set(
        [asset_helper(obj["href"]) for k, obj in stac_objects["assets"].items()]
    ) - set(remote_files)
    if diff:
        missing_files[stac_objects["id"]] = list(diff)
        scene_cnt += 1
        file_cnt += len(diff)

In [62]:
def retrieve_stac_from_S3(datestr):
    remote_stacs = [
        (obj.key, datestr + "/" + obj.key.split("/")[-1])
        for obj in my_bucket.objects.filter(Prefix=f"landsat9-c2-l1/{datestr}")
        if obj.key[-9:] == "stac.json"
    ]
    # retrieve stac json from remote S3 bucket if forgot to store a local copy
    for obj_key, obj_dest in remote_stacs:
        obj_dest = "../../data/stac/landsat-c2l1/json/" + obj_dest
        if not os.path.exists(obj_dest):
            if not os.path.exists(os.path.dirname(obj_dest)):
                os.makedirs(os.path.dirname(obj_dest))
            my_bucket.download_file(obj_key, obj_dest)

In [14]:
remote_stacs = [
    (obj.key, datestr + "/" + obj.key.split("/")[-1])
    for obj in my_bucket.objects.filter(Prefix="landsat9-c2-l1/20220926")
]

In [15]:
len(remote_stacs)

18379

In [ ]:
remote_stacs

In [78]:
retrieve_stac_from_S3("20221031")

In [61]:
start_date = date(2021, 10, 31)
end_date = date(2022, 11, 1)

In [ ]:
from glob import glob

glob(destination + "/*/")

In [66]:
source = "../../data/stac/landsat-c2l1/json/" + datestr
destination = "../../data/stac/landsat-c2l1/json/" + datestr

# code to move the files from sub-folder to main folder.
subfolders = os.listdir(source)

In [73]:
def move_and_clean_from_subfolder(datestr):
    import os
    import shutil
    from glob import glob

    # Define the source and destination path

    destination = "../../data/stac/landsat-c2l1/json/" + datestr
    tmp = "../../data/temp"
    source_folders = glob(destination + "/*/")
    # code to move the files from sub-folder to main folder.
    for source in source_folders:
        files = os.listdir(source)
        for file in files:
            file_name = os.path.join(source, file)
            shutil.move(file_name, tmp)
            shutil.rmtree(source)
            shutil.move(os.path.join(tmp, file), destination)

In [77]:
move_and_clean_from_subfolder("20221031")

In [ ]:
func_over_daterange(start_date, end_date, move_and_clean_from_subfolder)

In [ ]:
func_over_daterange(start_date, end_date, retrieve_stac_from_S3)

In [13]:
import glob

In [19]:
local_files = [
    fn.split("/")[-1]
    for fn in glob.glob("../../data/landsat/m2m_download/20220430/**/*")
]

In [20]:
len(local_files)

17232

In [35]:
len(remote_files)

17215

In [21]:
set(local_files) - set(remote_files)

{'LC09_L1TP_023017_20220430_20220430_02_T2_B4.TIF',
 'LC09_L1TP_071014_20220430_20220430_02_T1_B11.TIF',
 'LC09_L1TP_128057_20220430_20220430_02_T1_B10.TIF',
 'LC09_L1TP_144032_20220430_20220430_02_T1_QA_RADSAT.TIF',
 'LC09_L1TP_144032_20220430_20220430_02_T1_SAA.TIF',
 'LC09_L1TP_144033_20220430_20220430_02_T1_QA_RADSAT.TIF',
 'LC09_L1TP_144033_20220430_20220430_02_T1_SAA.TIF',
 'LC09_L1TP_144034_20220430_20220430_02_T1_QA_RADSAT.TIF',
 'LC09_L1TP_144034_20220430_20220430_02_T1_SAA.TIF',
 'LC09_L1TP_144034_20220430_20220430_02_T1_SZA.TIF',
 'LC09_L1TP_160012_20220430_20220430_02_T1_QA_PIXEL.TIF',
 'LC09_L1TP_176059_20220430_20220430_02_T1_SZA.TIF',
 'LC09_L1TP_176059_20220430_20220430_02_T1_VAA.TIF',
 'LC09_L1TP_192010_20220430_20220430_02_T1_SAA.TIF',
 'LC09_L1TP_192010_20220430_20220430_02_T1_SZA.TIF',
 'LC09_L1TP_192033_20220430_20220430_02_T1_QA_RADSAT.TIF',
 'LC09_L1TP_192033_20220430_20220430_02_T1_SAA.TIF'}

In [ ]:
my_config = Config(
    region_name="us-west-1",
    signature_version="v4",
    retries={"max_attempts": 10, "mode": "standard"},
)

In [ ]:
ACCESS_KEY = ""
SECRET_KEY = ""

PATH_TO_CSV = ""
car_dir = Path()

s3 = boto3.client(
    "s3",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    endpoint_url=ENDPOINT,
)

df = pd.read_csv(PATH_TO_CSV)


def upload_to_aws(local_file, bucket, s3_file):

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False


for idx, row in df.iterrows():
    print("Index ", idx)
    print(row["name"])
    car_target = car_dir / str(row["file"])

    if not car_target.exists():
        print(row["name"])
        print("does not exists")
        continue

    print(str(car_target), "<NAME>", row["file"])
    uploaded = upload_to_aws(str(car_target), "<NAME>", row["file"])

    if not uploaded:
        print(row["name"])
        print("not uploaded")